In [7]:
import pandas as pd
import numpy  as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

In [8]:
data=pd.read_csv("D:\python\Sales\sales_data_sample.csv",encoding="unicode_escape")

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2823 entries, 0 to 2822
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ORDERNUMBER       2823 non-null   int64  
 1   QUANTITYORDERED   2823 non-null   int64  
 2   PRICEEACH         2823 non-null   float64
 3   ORDERLINENUMBER   2823 non-null   int64  
 4   SALES             2823 non-null   float64
 5   ORDERDATE         2823 non-null   object 
 6   STATUS            2823 non-null   object 
 7   QTR_ID            2823 non-null   int64  
 8   MONTH_ID          2823 non-null   int64  
 9   YEAR_ID           2823 non-null   int64  
 10  PRODUCTLINE       2823 non-null   object 
 11  MSRP              2823 non-null   int64  
 12  PRODUCTCODE       2823 non-null   object 
 13  CUSTOMERNAME      2823 non-null   object 
 14  PHONE             2823 non-null   object 
 15  ADDRESSLINE1      2823 non-null   object 
 16  ADDRESSLINE2      302 non-null    object 


In [10]:
data.describe()

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,QTR_ID,MONTH_ID,YEAR_ID,MSRP
count,2823.000000,2823.000000,2823.000000,2823.000000,2823.000000,2823.000000,2823.000000,2823.00000,2823.000000
mean,10258.725115,35.092809,83.658544,6.466171,3553.889072,2.717676,7.092455,2003.81509,100.715551
std,92.085478,9.741443,20.174277,4.225841,1841.865106,1.203878,3.656633,0.69967,40.187912
min,10100.000000,6.000000,26.880000,1.000000,482.130000,1.000000,1.000000,2003.00000,33.000000
25%,10180.000000,27.000000,68.860000,3.000000,2203.430000,2.000000,4.000000,2003.00000,68.000000
50%,10262.000000,35.000000,95.700000,6.000000,3184.800000,3.000000,8.000000,2004.00000,99.000000
75%,10333.500000,43.000000,100.000000,9.000000,4508.000000,4.000000,11.000000,2004.00000,124.000000
max,10425.000000,97.000000,100.000000,18.000000,14082.800000,4.000000,12.000000,2005.00000,214.000000


In [11]:
temp=['CUSTOMERNAME','ORDERNUMBER',"ORDERDATE","SALES"]
RFM_data=data[temp]
RFM_data.shape

(2823, 4)

In [13]:
RFM_data["ORDERDATE"]=pd.to_datetime(RFM_data["ORDERDATE"])

<ipython-input-13-56c4616ce296>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  RFM_data["ORDERDATE"]=pd.to_datetime(RFM_data["ORDERDATE"])


In [14]:
RFM_data.head()

,CUSTOMERNAME,ORDERNUMBER,ORDERDATE,SALES
0,Land of Toys Inc.,10107,2003-02-24,2871.00
1,Reims Collectables,10121,2003-05-07,2765.90
2,Lyon Souveniers,10134,2003-07-01,3884.34
3,Toys4GrownUps.com,10145,2003-08-25,3746.70
4,Corporate Gift Ideas Co.,10159,2003-10-10,5205.27


In [15]:
RFM_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2823 entries, 0 to 2822
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   CUSTOMERNAME  2823 non-null   object        
 1   ORDERNUMBER   2823 non-null   int64         
 2   ORDERDATE     2823 non-null   datetime64[ns]
 3   SALES         2823 non-null   float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 88.3+ KB


In [16]:
RFM_data["ORDERDATE"].max()

Timestamp('2005-05-31 00:00:00')

In [17]:
NOW=RFM_data["ORDERDATE"].max()

In [18]:
RFM_table=RFM_data.groupby("CUSTOMERNAME").agg({"ORDERDATE": lambda x: (NOW-x.max()).days,
                                                "ORDERNUMBER":lambda x: len(x.unique()),
                                                "SALES":lambda x: x.sum()})

In [ ]:
RFM_table["ORDERDATE"]=RFM_table["ORDERDATE"].astype(int)

In [19]:
RFM_table.rename(columns={"ORDERDATE":"recency",
                          "ORDERNUMBER":"frequency",
                          "SALES":"monetry_value"}, inplace=True)
RFM_table.head()

,recency,frequency,monetry_value
CUSTOMERNAME,,,
"AV Stores, Co.",195,3,157807.81
Alpha Cognac,64,3,70488.44
Amica Models & Co.,264,2,94117.26
"Anna's Decorations, Ltd",83,4,153996.13
Atelier graphique,187,3,24179.96


In [23]:
quantiles=RFM_table.quantile(q=[0.25,0.5,0.75])
quantiles


,recency,frequency,monetry_value
0.25,80.25,2.0,70129.4325
0.50,185.00,3.0,86522.6100
0.75,229.25,3.0,120575.8750


In [24]:
quantiles=quantiles.to_dict()

In [45]:
RFM_segment=RFM_table.copy()

In [46]:
def R_Class (x,p,d):
    if x<=d[p][0.25]:
        return 4
    elif x<=d[p][0.5]:
        return 3
    elif x <=d[p][0.75]:
        return 2
    else:
        return 1
    
def FM_Class(x,p,d):
    if x<d[p][0.25]:
        return 1
    elif x<=d[p][0.5]:
        return 2
    elif x<=d[p][0.75]:
        return 3
    else:
        return 4

    

In [59]:
RFM_segment["R_Quartile"]=RFM_segment["recency"].apply(R_Class, args=("recency",quantiles))
RFM_segment["F_Quartile"]=RFM_segment["frequency"].apply(FM_Class,args=("frequency",quantiles))
RFM_segment["M_Quartile"]=RFM_segment["monetry_value"].apply(FM_Class,args=("monetry_value",quantiles))

RFM_segment["RFMClass"]=RFM_segment.R_Quartile.map(str)+RFM_segment.F_Quartile.map(str)+RFM_segment.M_Quartile.map(str)

BEST CUSTOMERS

In [65]:
RFM_segment[RFM_segment["RFMClass"]=="444"].sort_values("monetry_value", ascending=False).head()

RFM_segment.head()

,recency,frequency,monetry_value,R_Quartile,F_Quartile,M_Quartile,RFMClass
CUSTOMERNAME,,,,,,,
"AV Stores, Co.",195,3,157807.81,2,2,4,224
Alpha Cognac,64,3,70488.44,4,2,2,422
Amica Models & Co.,264,2,94117.26,1,2,3,123
"Anna's Decorations, Ltd",83,4,153996.13,3,4,4,344
Atelier graphique,187,3,24179.96,2,2,1,221


In [68]:
# low recency customers(customers at the verge of churning)
RFM_segment[RFM_segment["R_Quartile"]<=2].sort_values("monetry_value", ascending=False).head()

,recency,frequency,monetry_value,R_Quartile,F_Quartile,M_Quartile,RFMClass
CUSTOMERNAME,,,,,,,
Land of Toys Inc.,197,4,164069.44,2,4,4,244
"AV Stores, Co.",195,3,157807.81,2,2,4,224
"Saveley & Henriot, Co.",455,3,142874.25,1,2,4,124
Rovelli Gifts,200,3,137955.72,2,2,4,224
Online Diecast Creations Co.,208,3,131685.30,2,2,4,224


In [71]:
#Customers who's recency, frequency as well as monetary values are low (lost customers)
RFM_segment[RFM_segment["RFMClass"]=="111"].sort_values("recency",ascending=False).head()

,recency,frequency,monetry_value,R_Quartile,F_Quartile,M_Quartile,RFMClass
CUSTOMERNAME,,,,,,,
"Bavarian Collectables Imports, Co.",258,1,34993.92,1,1,1,111


In [72]:
# loyal customers
RFM_segment[RFM_segment["F_Quartile"]>=3].sort_values("monetry_value",ascending=False).head()

,recency,frequency,monetry_value,R_Quartile,F_Quartile,M_Quartile,RFMClass
CUSTOMERNAME,,,,,,,
Euro Shopping Channel,0,26,912294.11,4,4,4,444
Mini Gifts Distributors Ltd.,2,17,654858.06,4,4,4,444
"Australian Collectors, Co.",183,5,200995.41,3,4,4,344
Muscle Machine Inc,181,4,197736.94,3,4,4,344
La Rochelle Gifts,0,4,180124.90,4,4,4,444
